In [1]:
# Tiny Shakespeare download
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-10-25 08:57:32--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.06s   

2023-10-25 08:57:32 (18.3 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
# Datensatz einlesen und Parameter ausgeben
with open('input.txt', 'r', encoding='utf-8') as f:
  text = f.read()

In [6]:
print("Länge des Datensatzes: ", len(text), "\n")
print(text[:100])

Länge des Datensatzes:  1115394 

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [7]:
# Vokabular / Tokendomäne ermitteln

chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print("Vokabulargröße: ", vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
Vokabulargröße:  65


In [9]:
# Encoding und Decoding von chars zu ints und umgekehrt
# -----------------------------------------------------

# Abbildungslisten von chars auf Ints und umgekehrt
char_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_char = {i:ch for i,ch in enumerate(chars)}

# Encoding und Decoding mit den Listen
encode = lambda s: [char_to_int[c] for c in s]
decode = lambda l: [int_to_char[i] for i in l]

# Beispiel
print(encode("Fromage"))
print(decode(encode("Fromage")))

[18, 56, 53, 51, 39, 45, 43]
['F', 'r', 'o', 'm', 'a', 'g', 'e']


In [ ]:
# Jetzt torch-tauglich machen durch encoding als Tensor

import torch

# Den gesamten Shakespeare-Text als Tensor codieren
data = torch.tensor(encode(text), dtype=torch.long)

# Den Tensor angucken
print(data.shape, data.dtype)
print(data[:1000])

In [12]:
# Trennen von Trainings- und Validierungsdaten

n_split = int(0.9*len(data)) # Teilungsverhältnis (0.9)
train_data = data[:n_split]
val_data = data[n_split:]

In [13]:
# Aufteilen der Trainingsdaten in Blöcke für paralleles Training
block_size = 8
train_data[:block_size+1] # In diesem Tensor sind 8 Trainingsszenarien

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [14]:
# Zur Veranschaulichung
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f"Bei Input {context} ist die erwartete Ausgabe (das Target) {target}")

Bei Input tensor([18]) ist die erwartete Ausgabe (das Target) 47
Bei Input tensor([18, 47]) ist die erwartete Ausgabe (das Target) 56
Bei Input tensor([18, 47, 56]) ist die erwartete Ausgabe (das Target) 57
Bei Input tensor([18, 47, 56, 57]) ist die erwartete Ausgabe (das Target) 58
Bei Input tensor([18, 47, 56, 57, 58]) ist die erwartete Ausgabe (das Target) 1
Bei Input tensor([18, 47, 56, 57, 58,  1]) ist die erwartete Ausgabe (das Target) 15
Bei Input tensor([18, 47, 56, 57, 58,  1, 15]) ist die erwartete Ausgabe (das Target) 47
Bei Input tensor([18, 47, 56, 57, 58,  1, 15, 47]) ist die erwartete Ausgabe (das Target) 58


In [17]:
# Um das jetzt zu generalisieren, kann man smartere Tensoren bauen

torch.manual_seed(2424)
batch_size = 4 # Wie viele unabhängige Sequenzen parallel bearbeitet werden
block_size = 8 # Maximale Kontextlänge (siehe Oben)

def get_batch(split):
  # Je nachdem, ob Training oder Validation, zufällige Sequenz auswählen
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))

  # Die Tensoren füllen (Einmal Context, einmal Targets)
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])

  return x, y

# Ausgabe des Ergebnisses
# xb wird dann der Input für das neuronale Netz zum Lernen
xb, yb = get_batch('train')
print("Inputs: \n")
print("Input_Shape: ", xb.shape)
print(xb, "\n")
print("Targets: \n")
print("Target_Shape:", yb.shape)
print(yb)

Inputs: 

Input_Shape:  torch.Size([4, 8])
tensor([[57,  1, 47, 58,  1, 43,  5, 43],
        [63, 53, 59,  6,  1, 51, 63,  1],
        [53, 41, 50, 39, 47, 51,  5, 42],
        [57, 47, 56, 47, 52, 45,  1, 43]]) 

Targets: 

Target_Shape: torch.Size([4, 8])
tensor([[ 1, 47, 58,  1, 43,  5, 43, 52],
        [53, 59,  6,  1, 51, 63,  1, 50],
        [41, 50, 39, 47, 51,  5, 42,  1],
        [47, 56, 47, 52, 45,  1, 43, 63]])


In [29]:
# Einfachstes Beispiel -> BigramLanguageModel
# Idee: Lernen, welche Buchstaben direkt aufeinander folgen
# Also zum Beispiel: Wenn K, dann vermutlich a

import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(2424)

class BigramLanguageModel(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    # Aufbau einer Tabelle, in der die Aufeinanderfolgewahrscheinlichkeit steht
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, idx, targets=None):
    # Aufbau eines Tensors, der für jeden Batch die Blöcke mit den möglichen
    # Werten der Tabelle fusioniert, sodass am Ende (Batch x Times x Channels)
    # also 4 x 8 x 65 als Tensor entsteht -> Damit dann vorhersagen, für
    # welchen Char, welcher char als nächstes Kommt
    logits = self.token_embedding_table(idx)

    if targets is None:
      loss = None
    else:
      # Und weil (warum auch immer) das zunächst die falsche Form/Dimension hat:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)

      targets = targets.view(B*T)

      # Jetzt noch den Loss berechnen, um den Tensor zu updaten
      # Der Loss ist übrigens der Grund für den Dimensionskäse oben
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  # Jetzt kann man mit dem Modell auch etwas genereiren lassen
  def generate(self, idx, max_new_tokens):
    # idx ist das B x T Array aus Indices der Tokens
    for i in range(max_new_tokens):
      # Prediction vom Modell holen
      logits, loss = self(idx)
      # Fokus nur auf den letzten char (weil BigramLanguageModel)
      logits = logits[:, -1, :] # Nur noch B x C
      # Softmax nutzen, um Wahrscheinlichkeit zu ermitteln
      probs = F.softmax(logits, dim=-1)
      # Aus der ermittelten Verteilung ziehen
      idx_next = torch.multinomial(probs, num_samples=1) # Wird B x 1
      # Den ermittelten Int eines Chars an die Sequenz anhängen
      idx = torch.cat((idx, idx_next), dim=1) # Wird B X T+1
    return idx


m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape) # Das ist der Tensor mit B x T x C, aber runtergebrochen
print(loss)

# Test der Generatorfunktion
# Also: Dekodieren einer Sequenz aus einem 1x1 Tensor (das ist idx), der mit 0
# initialisiert ist und von generate mit max_new_tokens gefüllt wurde
generated = decode(m.generate(idx = torch.zeros((1,1), dtype=torch.long),
                        max_new_tokens = 100) [0].tolist())
print(''.join(generated))

torch.Size([32, 65])
tensor(4.7592, grad_fn=<NllLossBackward0>)

- s
Dg!IK
!rElmsfKUvdpIhbGg&o:ZiJOfKGqvF?bEdVcdoztaw:pRVkdp nUdBxHZJHEN!o,SJddpzIxJufp ZQJ'MUyudNCtn


In [39]:
# Jetzt gehts um basic learning, damit das Teil auch mal was rallt

# Optimizer erstellen
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

batch_size = 32
iterations = 10000

for steps in range(iterations):
  # Batch ziehen
  xb, yb = get_batch('train')

  # Loss berechnen und optimieren
  logits, loss = m(xb, yb)
  optimizer.zero_grad(set_to_none = True)
  loss.backward()
  optimizer.step()

#Ausgabe des letzten Losses
print(loss.item())

2.3895466327667236


In [40]:
# Und jetzt nochmal den Generator testen -> Hoffentlich deutlich besser
generated = decode(m.generate(idx = torch.zeros((1,1), dtype=torch.long),
                        max_new_tokens = 100) [0].tolist())
print(''.join(generated))


F thinot to thebeikive t at,
VOFowit my thortn, mavitineas stlorck t t youssourd n y.
'spridise that
